In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

* Following pytorch model has one linear layer with `input`: 3 and `output`:1

In [2]:
def create_model():
    class network(nn.Module):
        def __init__(self):
            super(network, self).__init__()
            self.fc1 = nn.Linear(3, 1)#100)
            #self.fc2 = nn.Linear(100, 100)
            #self.fc3 = nn.Linear(100, 1, bias=False)
            #self.softmax = nn.Softmax(dim=1)

        def forward(self, X):
            X = F.relu(self.fc1(X))
            #X = self.fc2(X)
            #X = self.fc3(X)
            #X = self.softmax(X)
            return X
    model= network()
    optimizer = optim.Adam(model.parameters())
    criterion= nn.MSELoss()# CrossEntropyLoss()
    return model, optimizer, criterion

In [3]:
model2, optimizer2, loss2= create_model()
model2

network(
  (fc1): Linear(in_features=3, out_features=1, bias=True)
)

* Data preparation scheme:

        - Each record `X` cosist of first 3 values of 4 values
          from iris dataset input features.
        
        - 4th value serve as taget label `y` .

In [4]:
def scale(arr):
    f= lambda x: (x-np.mean(x))/np.var(x)
    for idx in range(arr.shape[1]):
        arr[:,idx]= f(arr[:, idx])
    #return arr

In [5]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data#, data.target
scale(X)

trainx= Variable(torch.Tensor(X[:,:-1]).float(), requires_grad=True) #df.iloc[:, :-1].values).float()
trainy = Variable(torch.reshape(torch.Tensor(X[:,-1]).float(), (-1,1)))

#i_grads= np.empty(trainx.shape)
all_grads= list()
for epoch in range(128):
    optimizer2.zero_grad()
    y_pred = model2(trainx)
    #print(y_pred)
    l = loss2(y_pred, trainy)
    print('epoch: ', epoch, ' loss: ', l.item())
    # perform a backward pass (backpropagation)
    l.backward()
    wts_bias = list(model2.parameters())
    wts_bias_grads = torch.cat((wts_bias[0].grad, torch.reshape(wts_bias[1].grad, (-1,1))),1)
    #i_grads= trainx.grad
    all_grads.append(wts_bias_grads.tolist())#i_grads)
    # Update the parameters
    optimizer2.step()
all_grads = np.array(all_grads).reshape(-1,4)
all_grads_df= pd.DataFrame(all_grads, columns=['beta_{}'.format(idx) for idx in range(1,5)])

epoch:  0  loss:  1.8610870838165283
epoch:  1  loss:  1.8587082624435425
epoch:  2  loss:  1.8563390970230103
epoch:  3  loss:  1.8539788722991943
epoch:  4  loss:  1.8516281843185425
epoch:  5  loss:  1.849287748336792
epoch:  6  loss:  1.8469569683074951
epoch:  7  loss:  1.8446362018585205
epoch:  8  loss:  1.8423256874084473
epoch:  9  loss:  1.8400251865386963
epoch:  10  loss:  1.837735652923584
epoch:  11  loss:  1.835456132888794
epoch:  12  loss:  1.8331878185272217
epoch:  13  loss:  1.8309293985366821
epoch:  14  loss:  1.8286824226379395
epoch:  15  loss:  1.826446294784546
epoch:  16  loss:  1.8242213726043701
epoch:  17  loss:  1.8220078945159912
epoch:  18  loss:  1.8198052644729614
epoch:  19  loss:  1.8176140785217285
epoch:  20  loss:  1.815434217453003
epoch:  21  loss:  1.8132655620574951
epoch:  22  loss:  1.811109185218811
epoch:  23  loss:  1.808963418006897
epoch:  24  loss:  1.8068289756774902
epoch:  25  loss:  1.8047070503234863
epoch:  26  loss:  1.80259418

* Following are the weights and bias for the ONLY layer in model, after 128 epochs of training

In [7]:
li = list(model2.parameters())
li

[Parameter containing:
 tensor([[ 0.3514, -0.4006,  0.1464]], requires_grad=True),
 Parameter containing:
 tensor([-0.2750], requires_grad=True)]

In [8]:
print('Final weights gradients : {}\nFinal bias gradients: {}'.format(li[0].grad, li[1].grad))

Final weights gradients : tensor([[-0.5502, -0.9713, -0.2204]])
Final bias gradients: tensor([-0.1030])


* Following are gradients updated after each epochs

In [9]:
all_grads_df

,beta_1,beta_2,beta_3,beta_4
0,-0.569870,-1.490341,-0.170971,0.152157
1,-0.568760,-1.484230,-0.171056,0.150241
2,-0.567649,-1.478120,-0.171140,0.148325
3,-0.566538,-1.472011,-0.171224,0.146410
4,-0.565427,-1.465905,-0.171308,0.144496
...,...,...,...,...
123,-0.512670,-0.958388,-0.197748,-0.065333
124,-0.546060,-0.975163,-0.214689,-0.090420
125,-0.543684,-0.972776,-0.214009,-0.090020
126,-0.541277,-0.970449,-0.213312,-0.089577


* Following computes the values at gradients w.r.t all inputs `X`

In [10]:
optimizer2.zero_grad()
y_pred = model2(trainx)
l = loss2(y_pred, trainy)
l.backward()
i_grads= trainx.grad
#all_grads.append(i_grads)
#grad w.r.t input dataset Could be computed for every epoch

i_grads.shape

torch.Size([150, 3])

In [11]:
all_grads_df.describe()

,beta_1,beta_2,beta_3,beta_4
count,128.000000,128.000000,128.000000,128.000000
mean,-0.534177,-1.171829,-0.184331,0.034866
std,0.017789,0.161002,0.012604,0.071170
min,-0.569870,-1.490341,-0.220399,-0.102953
25%,-0.548793,-1.301782,-0.193073,-0.027634
50%,-0.534617,-1.147826,-0.185654,0.029436
75%,-0.520930,-1.020378,-0.171876,0.103084
max,-0.495673,-0.958388,-0.166877,0.152157


* Mean values for weight(Beta_1-3) and bias(beta_4) gradients 

In [12]:
all_grads_df.describe().iloc[1]

beta_1   -0.534177
beta_2   -1.171829
beta_3   -0.184331
beta_4    0.034866
Name: mean, dtype: float64

* Evaluating same data with linear regression from statsmodel

In [13]:
import statsmodels.api as sm
feats, targets= X[:,:-1], X[:,-1]
feats= sm.add_constant(feats, prepend=False)

In [14]:
model = sm.OLS(targets, feats, hasconst=True)
result= model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     734.4
Date:                Thu, 12 Mar 2020   Prob (F-statistic):           7.83e-88
Time:                        18:00:03   Log-Likelihood:                -45.701
No. Observations:                 150   AIC:                             99.40
Df Residuals:                     146   BIC:                             111.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2446      0.056     -4.363      0.0

* Weights and biases from modelstats regression model

In [15]:
result.params

array([-2.44611824e-01,  7.28612405e-02,  2.81096557e+00,  9.36750677e-17])

______________________

In [2]:
#def model():
#    class network(nn.Module):
#        def __init__(self):
#            super(network, self).__init__()
#            self.fc1 = nn.Linear(4, 100)
#            self.fc2 = nn.Linear(100, 100)
#            self.fc3 = nn.Linear(100, 3)
#            self.softmax = nn.Softmax(dim=1)
#
#        def forward(self, X):
#            X = F.relu(self.fc1(X))
#           X = self.fc2(X)
#            X = self.fc3(X)
            X = self.softmax(X)
#            return X
#    model= network()
#    optimizer = optim.Adam(model.parameters())
#    criterion= nn.CrossEntropyLoss()
#    return model, optimizer, criterion